# NYT Connections solver

### utils functionality demo

In [1]:
from scripts.puzzle_utils import Puzzle
from scripts.solver import Solver

puzzle = Puzzle()
random_solver = Solver()

# retrieve random puzzle from the database
random_words, id = puzzle.get_random_puzzle()
print(random_words)

# attempt to solve it (the example here is random ordering of words)
random_solver.solve_puzzle(random_words)
attempt_sol = random_solver.build_solution(random_words)
print(attempt_sol)

# check the solution
is_sol, num_corr = puzzle.check_solution(attempt_sol, id)
print(f"Is solution? {is_sol} \nNumber of matches: {num_corr}")

['SENTENCE', 'PIPE', 'PASSAGE', 'JUG', 'PHRASE', 'SANDS', 'NICK', 'RECORDER', 'CRACK', 'WHISTLE', 'WASTE', 'PUNCTURE', 'LETTER', 'WORD', 'HOLE', 'LEAK']
[{'PUNCTURE', 'SENTENCE', 'PHRASE', 'LEAK'}, {'PASSAGE', 'NICK', 'LETTER', 'PIPE'}, {'JUG', 'SANDS', 'RECORDER', 'WORD'}, {'WHISTLE', 'WASTE', 'CRACK', 'HOLE'}]
Is solution? False 
Number of matches: 6


In [2]:
from scripts.glove import GloVe

glove = GloVe("/home/hether/coding_projects/nyt-connections-solver/embeddings/glove.6B.300d.txt")

Creating GloVe lookup map: 400000it [00:08, 46642.15it/s]


In [3]:
import numpy as np
print(glove.embed_puzzle_words(random_words))

[[-0.32636  -0.46357  -0.75916  ...  0.058154 -0.48012   0.053241]
 [ 0.17264   0.81223  -0.32949  ...  0.048746 -0.45891   0.39119 ]
 [ 0.018861 -0.71351  -0.15123  ... -0.14436  -0.47893  -0.003815]
 ...
 [-0.43036  -0.40622  -0.18279  ...  0.028737 -0.42117   0.27851 ]
 [-0.28281   0.18745  -0.59173  ... -0.36031  -0.037287 -0.17796 ]
 [ 0.33893   0.48199  -0.15186  ... -0.54716  -0.40129   0.68783 ]]


### k-means approach

In [4]:
from scripts.kmeans import KMeansSolver

# retrieve random puzzle from the database
random_words, id = puzzle.get_random_puzzle()
print(random_words)

# attempt to solve it (fixed size kmeans)
kmeans_solver = KMeansSolver(glove)
kmeans_solver.solve_puzzle(random_words)
attempt_sol = kmeans_solver.build_solution(random_words)
print(attempt_sol)

# check the solution
is_sol, num_corr = puzzle.check_solution(attempt_sol, id)
print(f"Is solution? {is_sol} \nNumber of matches: {num_corr}")

['PELICAN', 'BEACH', 'PUFFIN', 'SUNFLOWER', 'IRATE', 'BOOBY', 'SCHOOLBUS', 'BANANA', 'INCENSED', 'FURIOUS', 'HARDY', 'LIVID', 'CANARY', 'GULL', 'BEASTIE', 'LOST']
[{'FURIOUS', 'LIVID', 'INCENSED', 'IRATE'}, {'BOOBY', 'GULL', 'CANARY', 'SUNFLOWER'}, {'PELICAN', 'PUFFIN', 'BEASTIE', 'SCHOOLBUS'}, {'HARDY', 'BEACH', 'BANANA', 'LOST'}]
Is solution? False 
Number of matches: 11


### testing approaches

In [25]:
from scripts.tester import MethodTester

tester = MethodTester()
results_random = tester.test_solver(random_solver)
results_kmeans = tester.test_solver(kmeans_solver)

print(f"Random Solver:\nNumber of solutions: {np.sum(results_random['is_sol'])}/638\nAverage number of correct words: {np.mean(results_random['num_corr'])}/16")
print(f"K-means Solver:\nNumber of solutions: {np.sum(results_kmeans['is_sol'])}/638\nAverage number of correct words: {np.mean(results_kmeans['num_corr'])}/16")

100%|██████████| 638/638 [00:00<00:00, 2086.84it/s]

Random Solver:
Number of solutions: 0/638
Average number of correct words: 5.650470219435737/16
K-means Solver:
Number of solutions: 8/638
Average number of correct words: 7.89068100358423/16
